In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import time
import os
import gc ## gabage collector

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import LSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *

**Load the training and testing dataset**

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
#train_df['question_text'][train_df['target']==1]

In [ ]:
embed_size = 300 
vocab_size = 95000  # number of unique words in corpus
input_size = 70 # length of input questions

In [ ]:
## fill up missing values
filled_train = train_df["question_text"].fillna("_##_").values
filled_test = test_df["question_text"].fillna("_##_").values

## Tokenise
t = Tokenizer(num_words = vocab_size)
t.fit_on_texts(list(filled_train))

## encode the sequences
encoded_train = t.texts_to_sequences(filled_train)
encoded_test = t.texts_to_sequences(filled_test)

## pad the sequences
padded_train = pad_sequences(encoded_train, maxlen=input_size)
padded_test = pad_sequences(encoded_test, maxlen=input_size)

## labels
labels = train_df["target"].values

## shuffle the dataset
np.random.seed(2018)
shuffled_index = np.random.permutation(len(padded_train))
train = padded_train[shuffled_index]
labels = labels[shuffled_index]

**Load the embedding GloVe**


In [ ]:
def load_embedding(url):
    embeddings_index = dict()
    f = open(url)
    for line in f:
        values = line.split(" ")
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()    
    embedding_matrix = np.zeros((vocab_size, embed_size))
    for word, i in t.word_index.items():
        if i >= vocab_size: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    ## release memory
    del embeddings_index
    import gc; gc.collect()
    time.sleep(10)
    return embedding_matrix

In [ ]:
glove_matrix = load_embedding('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
#wiki_matrix = load_embedding('../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec')
#para_matrix = load_embedding('../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt')
#embedding_matrix = np.mean([glove_matrix, para_matrix], axis = 0)
embedding_matrix = glove_matrix

**Building the model**

In [ ]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
## Input layer
inp = Input(shape=(input_size,))

## Embedding layer with dropout of 20 percents
x = Embedding(vocab_size, embed_size, weights=[embedding_matrix], input_length=input_size, trainable=False)(inp)
x = SpatialDropout1D(0.2)(x)

## LSTM
lstm = Bidirectional(CuDNNLSTM(input_size, return_sequences=True))(x)
atten_1 = Attention(input_size)(lstm)
max_pool_1 = GlobalMaxPooling1D()(lstm)
#conc_1 = concatenate([max_pool_1, atten_1])
#dropout_1 = Dropout(0.1)(max_pool_1)
#dense_1 = Dense(input_size, activation='relu')(dropout_1)

## GRU Layer
gru = Bidirectional(CuDNNGRU(input_size, return_sequences=True))(x)
atten_2 = Attention(input_size)(gru)
max_pool_2 = GlobalMaxPooling1D()(gru)
#conc_2 = concatenate([atten_2, max_pool_2])
#dropout_2 = Dropout(0.1)(max_pool_2)
#dense_2 = Dense(input_size, activation='relu')(dropout_2)

conc1 = concatenate([atten_1, atten_2])
#conc1 = GlobalMaxPooling1D()(conc1)
conc1 = Dropout(0.1)(conc1)

conc2 = concatenate([max_pool_1,max_pool_2])
#conc2 = GlobalMaxPooling1D()(conc2)
conc2 = Dropout(0.1)(conc2)


conv1 = Conv1D(128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
#conv2 = GlobalMaxPooling1D()(conv1)
conv3 = GlobalAveragePooling1D()(conv1)

conc = concatenate([conc1, conc2, conv3])
conc = Dropout(0.2)(conc)
conc = Dense(input_size, activation='relu')(conc)
conc = Dropout(0.1)(conc)
outp = Dense(1, activation='sigmoid')(conc)

model = Model(inputs = inp, outputs = outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])

# summarize the model
print(model.summary())

In [ ]:
permuted_index = np.random.permutation(len(train))
percentage = 0.2 # percentage of data used for validation
cutoff = int(len(train)*percentage)
train_index = permuted_index[cutoff:]
val_index = permuted_index[:cutoff]

## training and labels
trains = train[train_index]
train_labels = labels[train_index]

## validation set
val = train[val_index]
val_labels = labels[val_index]

print(len(train))
print(len(trains)+len(val))
print(len(train_labels)+len(val_labels))

In [ ]:
# fit the model
model.fit(trains, train_labels, batch_size=512, epochs=20, verbose=1, callbacks=None, validation_data=(val, val_labels), shuffle=True)

In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(val, val_labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

**Choose threshold **

In [ ]:
pred_val = model.predict([val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = f1_score(val_labels, (pred_val>thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
 
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

**Run on test data**

In [ ]:
pred_test = model.predict(padded_test, batch_size=1024, verbose=1)

In [ ]:
a = []
for line in pred_test:
    a.append((line >= best_thresh).astype(int)[0])

In [ ]:
test_df.head()

In [ ]:
#sub = pd.read_csv('../input/sample_submission.csv')
sub = pd.DataFrame()
sub['qid'] = test_df['qid'].values
sub['prediction'] = a
sub.to_csv("submission.csv", index=False)
sub.head()